In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

project_name = 'finetune_para_wordembed'
%env WANDB_PROJECT=$project_name

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=finetune_para_wordembed


### Karger Books Para

In [11]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [12]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/bart/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_karger_books_para_wordembed_train/"'

log_dir = bucket_dir + '/logs'

In [9]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path, use_cache=False)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.length_penalty = config.LENGTH_PENALTY
model_config.no_repeat_ngram_size = config.NO_REPEAT_NGRAM_SIZE

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['length_penalty'] = config.LENGTH_PENALTY
model_config.task_specific_params['summarization']['no_repeat_ngram_size'] = config.NO_REPEAT_NGRAM_SIZE
model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

404 Client Error: Not Found for url: https://huggingface.co/%22/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_traincheckpoint-325%22/resolve/main/config.json


OSError: Can't load config for '"/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_traincheckpoint-325"'. Make sure that:

- '"/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_traincheckpoint-325"' is a correct model identifier listed on 'https://huggingface.co/models'

- or '"/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_traincheckpoint-325"' is the correct path to a directory containing a config.json file



##### Fine tune

In [13]:
model_name_or_path = output_dir[:-1] + 'checkpoint-325"'
model_name_or_path

'"/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-325"'

In [14]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6', use_cache=False)
tok.save_pretrained(model_name_or_path[1:-1])

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

('/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-325/tokenizer_config.json',
 '/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-325/special_tokens_map.json',
 '/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-325/vocab.json',
 '/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-325/merges.txt',
 '/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-325/added_tokens.json')

In [15]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--fp16 \
--learning_rate 3e-5 --label_smoothing 0.1 \
--freeze_embeds --freeze_encoder \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--test_max_target_length $config.ONE_BULLET_MAX_LEN \
--do_train \
--num_train_epochs 5 \
--logging_steps 10 --save_steps 100 --save_total_limit 3 \
--per_device_train_batch_size 16 --per_device_eval_batch_size 16 --gradient_accumulation_steps 1 \
--do_eval --evaluation_strategy steps --eval_steps 25 --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--load_best_model_at_end \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

01/22/2021 11:53:54 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
01/22/2021 11:53:54 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Jan22_11-53-54_ip-172-31-39-35', logging_first_step=False, logging_steps=10, save_steps=100, save_total_li

01/22/2021 11:54:50 - INFO - __main__ -   *** Train ***
[INFO|trainer.py:703] 2021-01-22 11:55:21,580 >> ***** Running training *****
[INFO|trainer.py:704] 2021-01-22 11:55:21,580 >>   Num examples = 2046
[INFO|trainer.py:705] 2021-01-22 11:55:21,580 >>   Num Epochs = 5
[INFO|trainer.py:706] 2021-01-22 11:55:21,580 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:707] 2021-01-22 11:55:21,580 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:708] 2021-01-22 11:55:21,580 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:709] 2021-01-22 11:55:21,580 >>   Total optimization steps = 640
[INFO|trainer.py:725] 2021-01-22 11:55:21,619 >>   Continuing training from checkpoint, will skip to saved global_step
[INFO|trainer.py:726] 2021-01-22 11:55:21,619 >>   Continuing training from epoch 2
[INFO|trainer.py:727] 2021-01-22 11:55:21,619 >>   Continuing training from global step 325
[INFO|trainer.py:730] 2021-01-22 11:55:21,619 >>   Will 

 94%|█████████▍| 16/17 [03:03<00:10, 10.99s/it]
                                                 {'eval_loss': 2083.360107421875, 'eval_rouge1': 32.0908, 'eval_rouge2': 13.5278, 'eval_rougeL': 25.7769, 'eval_rougeLsum': 26.3739, 'eval_gen_len': 35.1, 'epoch': 3.125}
100%|██████████| 17/17 [03:16<00:00, 11.32s/it]
                                               
[INFO|trainer.py:1226] 2021-01-22 12:09:16,909 >> Saving model checkpoint to /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-400
[INFO|configuration_utils.py:289] 2021-01-22 12:09:17,062 >> Configuration saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-400/config.json
[INFO|modeling_utils.py:814] 2021-01-22 12:10:01,822 >> Model weights saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-400/pytorch_model.bin
[INFO|trainer.py:1285] 2021-01-22 12:10:36,969

[INFO|configuration_utils.py:289] 2021-01-22 12:24:55,551 >> Configuration saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-475/config.json
[INFO|modeling_utils.py:814] 2021-01-22 12:25:41,974 >> Model weights saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-475/pytorch_model.bin
[INFO|trainer.py:1285] 2021-01-22 12:26:11,372 >> Deleting older checkpoint [/home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-475] due to args.save_total_limit
{'loss': 1628.46650390625, 'learning_rate': 7.5e-06, 'epoch': 3.75}
                                                 {'loss': 1957.0677734375, 'learning_rate': 7.03125e-06, 'epoch': 3.828125}
 78%|███████▊  | 500/640 [30:59<00:50,  2.77it/s][INFO|trainer.py:1412] 2021-01-22 12:26:22,375 >> ***** Running Evaluation *****
[INFO|trainer.py:1413] 2021-01-22 12:26:22,375 >>   Num e

 90%|████████▉ | 575/640 [46:26<00:23,  2.79it/s][INFO|trainer.py:1412] 2021-01-22 12:41:49,677 >> ***** Running Evaluation *****
[INFO|trainer.py:1413] 2021-01-22 12:41:49,677 >>   Num examples = 266
[INFO|trainer.py:1414] 2021-01-22 12:41:49,677 >>   Batch size = 16

 94%|█████████▍| 16/17 [03:08<00:10, 10.99s/it]
                                                 A
{'eval_loss': 2081.237060546875, 'eval_rouge1': 31.7977, 'eval_rouge2': 13.4768, 'eval_rougeL': 25.2941, 'eval_rougeLsum': 25.898, 'eval_gen_len': 34.6, 'epoch': 4.4921875}
100%|██████████| 17/17 [03:21<00:00, 11.13s/it]
                                               [INFO|trainer.py:1226] 2021-01-22 12:45:30,410 >> Saving model checkpoint to /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-575
[INFO|configuration_utils.py:289] 2021-01-22 12:45:30,587 >> Configuration saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/che

[INFO|modeling_utils.py:1140] 2021-01-22 12:57:58,569 >> All model checkpoint weights were used when initializing BartForConditionalGeneration.

[INFO|modeling_utils.py:1149] 2021-01-22 12:57:58,569 >> All the weights of BartForConditionalGeneration were initialized from the model checkpoint at /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/checkpoint-275.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
{'epoch': 5.0}                                     
100%|██████████| 640/640 [1:02:35<00:00,  5.87s/it]
[INFO|trainer.py:1226] 2021-01-22 12:57:59,375 >> Saving model checkpoint to /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_wordembed_train/
[INFO|configuration_utils.py:289] 2021-01-22 12:57:59,535 >> Configuration saved in /home/ubuntu/s3/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_bo

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--bs 2 \
--length_penalty $config.LENAGTH_PENALTY \
--no_repeat_ngram_size $config.NO_REPEAT_NGRAM_SIZE \
--num_beams $config.NUM_BEAMS \
--dump-args